In [1]:
import pandas as pd
import datetime
from configparser import ConfigParser
import json, requests, datetime
from pysimplicate import Simplicate

# Pandas
pd.options.display.float_format = '{:,.1f}'.format
pd.set_option('display.max_columns', 500)
PANDAS_FILE = '../simplicate_cache/hours.pd'
df = pd.read_pickle(PANDAS_FILE)

df = df.query( 'type=="normal"')
df['turnover'] = df.apply(lambda a: (a['hours']+a['corrections']) * (a['tariff'] if a['tariff'] > 0 else a['service_tariff']), axis=1)
df['turnover'] = df.apply(lambda a: a['turnover']/2 if a['project_number'] == 'TOR-3' else a['turnover'], axis=1)
df['week'] = df.apply(lambda a: datetime.datetime.strptime(a['day'],'%Y-%m-%d').isocalendar()[1], axis=1)
df['month'] = df.apply(lambda a: datetime.datetime.strptime(a['day'],'%Y-%m-%d').month, axis=1)
df['corrections_value'] = df.apply(lambda a: (a['corrections']) * (a['tariff'] if a['tariff'] > 0 else a['service_tariff']), axis=1)

# Simplicate
ini = ConfigParser()
ini.read('../credentials.ini')

subdomain = ini['simplicate']['subdomain']
api_key = ini['simplicate']['api_key']
api_secret = ini['simplicate']['api_secret']

sim = Simplicate(subdomain, api_key, api_secret )

In [2]:
df[-3:]

,employee,organization,project_id,project_name,project_number,service,type,service_tariff,label,billable,tariff,hours,day,status,corrections,turnover,week,corrections_value,month
13386,Mandy Dorée,Oberon,project:21ea648f0c0c31fcfeaad60b7a7437df,Internal,OBE-1,Other / Unaccountable,normal,0.0,Internal,False,0.0,0.5,2021-05-10,to_forward,0.0,0.0,19,0.0,5
13387,Mandy Dorée,Oberon,project:21ea648f0c0c31fcfeaad60b7a7437df,Internal,OBE-1,Other / Unaccountable,normal,0.0,Internal,False,0.0,0.2,2021-05-10,to_forward,0.0,0.0,19,0.0,5
13388,Mandy Dorée,TOR groep,project:5fdd3f799c981c6dfeaad60b7a7437df,TOR 3.0,TOR-3,Development Sprints Q1&Q2,normal,80.0,Testing,False,77.5,0.2,2021-05-10,to_forward,0.0,9.7,19,0.0,5


## Turnover

In [3]:
def turnover( project, month=None, from_date=None ):
    query = f'project_number=="{project}"'
    if month: 
        query += f' and month=={month}'
    if from_date:
        query += f' and day>="{from_date}"'
    data = df.query(query)
    return data['turnover'].sum()

turnover( 'BAM-1', from_date='2021-02-01')

86545.15000000001

## Invoices

In [4]:
def invoiced(project, month=None ):
    filter = {'project_number':project}
    if month:
        filter['from_date'] = f'2021-0{month}-01'
        filter['until_date'] = f'2021-0{month+1}-01'
    invoices = sim.invoice( filter )
    tot = 0
    for invoice in invoices:
        #print( invoice.get('invoice_number','????'), invoice['total_excluding_vat'], invoice['status'])
        tot += invoice['total_excluding_vat']
    return tot

def last_invoice_date( project ):
    invoices = sim.invoice( {'project_number':project} )
    invoices = sorted( invoices, key=lambda i: i['date'])
    if invoices:
        return datetime.datetime.strptime( invoices[-1]['date'], '%Y-%m-%d').date()
    
#print( invoiced( 'BAM-1'))
print( last_invoice_date( 'SLIM-16' ))

2020-11-30


## All projects, certain month

In [5]:
def active_projects():
    projects = [{'project': project.get('project_number',''),
                 'spent' : project['budget']['hours'].get('value_spent', 0),
                 'invoiced' : project['budget']['total']['value_invoiced']
                }
                for project in sim.project( {'active':True} )]
    return projects
active_projects()[:3]

[{'project': 'TRAV-3', 'spent': 0, 'invoiced': 0},
 {'project': 'SM2021', 'spent': 70018.75, 'invoiced': 85170},
 {'project': 'VOLK-1', 'spent': 12672, 'invoiced': 11724}]

## Onderhanden werk

In [6]:
pd.set_option('display.max_row', 150)

# project['budget']['total']['value_invoiced']
def corrections(p):
    project = p['project']
    h = df.query( f'project_number=="{project}"' )
    print( 'corrections', project, h['corrections_value'].sum())
    return h['corrections_value'].sum()
    
def onderhanden():
    return pd.DataFrame( [{'project':project['project'], 
                           'spent':project['spent'],
                           'corr': corrections( project),
                           'inv': project['invoiced'],
                           'OH':project['spent'] + corrections( project) - project['invoiced']} 
                          for project in active_projects()] ).sort_values( by=['OH'])
oh = onderhanden()
oh.drop( oh[(oh.project=='TOR-3')].index, inplace=True)
oh

corrections TRAV-3 0.0
corrections TRAV-3 0.0
corrections SM2021 340.0
corrections SM2021 340.0
corrections VOLK-1 -3910.5
corrections VOLK-1 -3910.5
corrections EUR-1 -7334.2499999967
corrections EUR-1 -7334.2499999967
corrections HAVA-2 -8585.0
corrections HAVA-2 -8585.0
corrections OBE-1 0.0
corrections OBE-1 0.0
corrections AME-1 0.0
corrections AME-1 0.0
corrections TOR-3 -14064.375
corrections TOR-3 -14064.375
corrections TEX-1 -58967.5
corrections TEX-1 -58967.5
corrections ACC-1 0.0
corrections ACC-1 0.0
corrections THIE-23 0.0
corrections THIE-23 0.0
corrections TOMMY-1 -810.0
corrections TOMMY-1 -810.0
corrections BOER-1 -315.0
corrections BOER-1 -315.0
corrections TOPA-1 55.0
corrections TOPA-1 55.0
corrections ESCH-1 -5720.0
corrections ESCH-1 -5720.0
corrections KPITO-1 -82.5
corrections KPITO-1 -82.5
corrections BREI-1 0.0
corrections BREI-1 0.0
corrections BAM-1 -3188.875
corrections BAM-1 -3188.875
corrections RIN-1 -750.0
corrections RIN-1 -750.0
corrections TRAV-4 0.0

,project,spent,corr,inv,OH
1,SM2021,"70,018.8",340.0,"85,170.0","-14,811.2"
48,TEX-2,"2,617.5",-585.0,"12,372.5","-10,340.0"
34,MTM-1,"58,150.6",0.0,"67,454.4","-9,303.8"
14,ESCH-1,"10,402.5","-5,720.0","12,752.5","-8,070.0"
42,TMC-1,"17,627.5","-2,612.5","21,440.0","-6,425.0"
49,LEAN-2,0.0,0.0,"5,950.0","-5,950.0"
6,AME-1,137.5,0.0,"5,897.5","-5,760.0"
88,ONC-2,"5,185.0",0.0,"10,200.0","-5,015.0"
30,BITF-1,"6,375.0",0.0,"10,880.0","-4,505.0"
82,BAM-2,0.0,0.0,"4,200.0","-4,200.0"


In [7]:
oh['OH'].sum()

21726.7824670033

# Correcties

In [8]:
df[-3:]

,employee,organization,project_id,project_name,project_number,service,type,service_tariff,label,billable,tariff,hours,day,status,corrections,turnover,week,corrections_value,month
13386,Mandy Dorée,Oberon,project:21ea648f0c0c31fcfeaad60b7a7437df,Internal,OBE-1,Other / Unaccountable,normal,0.0,Internal,False,0.0,0.5,2021-05-10,to_forward,0.0,0.0,19,0.0,5
13387,Mandy Dorée,Oberon,project:21ea648f0c0c31fcfeaad60b7a7437df,Internal,OBE-1,Other / Unaccountable,normal,0.0,Internal,False,0.0,0.2,2021-05-10,to_forward,0.0,0.0,19,0.0,5
13388,Mandy Dorée,TOR groep,project:5fdd3f799c981c6dfeaad60b7a7437df,TOR 3.0,TOR-3,Development Sprints Q1&Q2,normal,80.0,Testing,False,77.5,0.2,2021-05-10,to_forward,0.0,9.7,19,0.0,5


In [9]:
import datetime
one_week_ago = (datetime.datetime.today() + datetime.timedelta(weeks=-1)).strftime(DATE_FORMAT)
five_weeks_ago = (datetime.datetime.today() + datetime.timedelta(weeks=-5)).strftime(DATE_FORMAT)
data = df.query(f'(tariff>0 or service_tariff>0) and day>="{one_week_ago}" and day<"{five_weeks_ago}"')
data

NameError: name 'DATE_FORMAT' is not defined

In [ ]:
df['day'].min()

In [ ]:
one_week_ago = (datetime.datetime.today() + datetime.timedelta(weeks=-1)).strftime(DATE_FORMAT)
five_weeks_ago = (datetime.datetime.today() + datetime.timedelta(weeks=-5)).strftime(DATE_FORMAT)
query = f'(tariff>0 or service_tariff>0) and day>="{five_weeks_ago}" and day<"{one_week_ago}"'
print( query )
data = df.query(query)
print( data['day'].min() )
print( data['day'].max() )
print( data['hours'].sum() )
percentage_corrected = 100 * -data['corrections'].sum() / data['hours'].sum()
percentage_corrected

In [ ]:
DATE_FORMAT = '%Y-%m-%d'
lastmonth = (datetime.datetime.today() + datetime.timedelta(days=-30)).strftime(DATE_FORMAT)
a = df.query(f'corrections < 0 and day>="{lastmonth}"').groupby(['organization','project_name']).agg({'hours':'sum','corrections':'sum', 'turnover':'sum'}).sort_values('corrections').query('corrections < -10')
a.reset_index()

## Correcties op een specifiek project

In [ ]:
easy = df.query('project_number == "EASY-1"')
easy

In [ ]:
easy.groupby(['organization','project_name','project_id']).agg({'hours':'sum','corrections':'sum', 'turnover':'sum'}).query('corrections < -10')

In [ ]:
df.query(f'corrections < 0 and project_number == "EASY-1"')\
        .groupby(['organization', 'project_name', 'project_id'])\
        .agg({'hours': 'sum', 'corrections': 'sum'})\
        .sort_values('corrections')\
        .reset_index()

In [ ]:
df.query('corrections < 0').query('project_number == "EASY-1"').groupby(['organization','project_name','project_id']).agg({'hours':'sum','corrections':'sum', 'turnover':'sum'}).query('corrections < -10')

In [ ]:
df.query('corrections < 0 and project_number == "EASY-1"')